In [10]:
import pandas as pd
import numpy as np
import os
import tqdm
import shutil

In [11]:
dataset = 'onion' # onion or emma
input_data_dir = f'../data/'

features = {
    'emotion_embeddings': ['id_emma.tsv', 'id_gems.tsv'],
    'audio_embeddings': ['id_maest.tsv'],# 'id_jukebox.tsv', 'id_musicnn.tsv'],
    'textual_embeddings': ['id_bert.tsv'],# 'id_lyrics_word2vec.tsv'],
    'visual_embeddings': ['id_resnet.tsv'],# 'id_vgg19.tsv', 'id_incp.tsv'],
}

output_dir = f'../dataset/{dataset}/'

os.makedirs(output_dir, exist_ok=True)

In [12]:
df_interactions = pd.read_csv(f'../data/userid_trackid_timestamp_{dataset}.tsv', sep='\t')
    
df_interactions['rating'] = 5
sample_items = df_interactions['track_id'].unique()

# map item and users to numbers
df_interactions['user_id_int'] = df_interactions['user_id'].astype('category').cat.codes
df_interactions['track_id_int'] = df_interactions['track_id'].astype('category').cat.codes


print(df_interactions.shape)
df_interactions.head()

(306338, 8)


,user_id,track_id,timestamp,year,month,rating,user_id_int,track_id_int
0,31718,tc3VSkQnxAcVgtJ0,2014-05-14 05:48:36,2014,2014-05,5,11923,27517
1,44824,tecON02DYeSrxsgw,2014-05-20 04:29:32,2014,2014-05,5,15392,27539
2,50298,ZS7cYFuaZpVPjcCa,2014-05-11 16:04:43,2014,2014-05,5,16820,17378
3,68451,BLQ0MVtFsdqusuoC,2014-05-08 17:21:00,2014,2014-05,5,20924,5663
4,113497,wLrYaZGgflCudkS2,2014-05-14 23:26:42,2014,2014-05,5,27060,28882


In [13]:
# track ids from feature file
df = pd.read_csv(input_data_dir + features['emotion_embeddings'][1], sep='\t')
feat_track_ids = df['id'].values
feat_track_ids

array(['04q3VppIQEET5rzy', '07xF9Q0K1t3ist7K', '0JbYcELIqpMPvAHk', ...,
       'zzyyPUs7hC9Nz2e1', 'zzz0n04uuTUA7fNh', 'zzzj3LYaZtYtbzSr'],
      dtype=object)

In [14]:
# remove interactions with items not in the feature files
df_interactions = df_interactions[df_interactions['track_id'].isin(feat_track_ids)]
df_interactions.shape

(306338, 8)

In [15]:
#df_interactions[['user_id', 'track_id', 'rating', 'timestamp']].to_csv(os.path.join(output_dir, 'interactions.tsv'), index=False, sep='\t', header=False)
df_interactions[['user_id_int', 'track_id_int', 'rating', 'timestamp']].to_csv(os.path.join(output_dir, 'interactions.tsv'), index=False, sep='\t', header=False)

In [16]:
item_id_map = df_interactions[['track_id', 'track_id_int']].drop_duplicates()
item_id_map.set_index('track_id', inplace=True)
item_id_map.head()

,track_id_int
track_id,
tc3VSkQnxAcVgtJ0,27517
tecON02DYeSrxsgw,27539
ZS7cYFuaZpVPjcCa,17378
BLQ0MVtFsdqusuoC,5663
wLrYaZGgflCudkS2,28882


In [17]:
del df_interactions

In [ ]:
for feature_type in features:

    for file in features[feature_type]:
        if dataset == 'onion' and 'emma' in file:
            continue

        print('Processing', file)
        df = pd.read_csv(os.path.join(input_data_dir, file), index_col=0, sep='\t')
        df = df.merge(item_id_map, left_index=True, right_index=True)
        feature_name = file.split('.')[0].split('_')[1]
            
        folder_path = os.path.join(output_dir, feature_type, feature_name)

        if os.path.exists(folder_path):
            shutil.rmtree(folder_path)
        os.makedirs(folder_path, exist_ok=True)

        for index, row in tqdm.tqdm(df.iterrows()):
            np.save(os.path.join(folder_path, str(int(row['track_id_int'])) + '.npy'), row.values[:-1])

Processing id_gems.tsv


19357it [00:55, 344.97it/s]